In [ ]:
import kenlm

model = kenlm.Model('../../kenlm/char_5gram_ru_en.bin')

def tok(t):
    return ' '.join(['<space>' if j == ' ' else j for j in t])

In [ ]:
import pandas as pd

df = pd.read_parquet('name_desc_avito.parquet')

In [ ]:
import re

df_name_eng = df['name'][df['name'].str.match('[a-zA-Z]')]
df_name_eng

6                                    iPhone 11, 64 ГБ
13                             iPhone 13 mini, 256 ГБ
16                  Frederic malle french rover 10 ml
37                                  PS4 Интеpaктивнoе
45                   Eva/Эвa/Евa ковpики в aвтомобиль
                              ...                    
3263497       TTP111VPC Пpиемопеpедaтчик видеоcигнaлa
3263514                         iPhone 15 Pro, 128 ГБ
3263541                             Nokia type: CUW-2
3263543                        iPhone 15 Plus, 128 ГБ
3263548    My BMW Connected Drive Пpиложение Подпиcкa
Name: name, Length: 375464, dtype: object

In [291]:
df_desc_eng = df['description'][df['description'].str.match('[a-zA-Z]')]
df_desc_eng

6          iPhone 11 64gb\n\nЕмкоcть АКБ 87%\n\nФейc paбо...
29         Denon AH-D1200 — Идеaльные пoлнopaзмеpные нaуш...
37         The Dark Pictures Anthology: \r\n-Little Hope ...
42         pегулиpуемый paзмеp кольцa. cтеклышко ~20мм (ф...
43         Sakura SASP377XE Кapтpидж cовмеcтимый Сaкуpa S...
                                 ...                        
3263531    CET CET7906 Вaл pезиновый 022N02374 Pressure R...
3263541                                    Nokia type: CUW-2
3263543    iPhоne 15 Plus, 128 GB, Pink. Новый, зaпeчaтaн...
3263554    Cactus CS-C731M (987395)\n\nСaмовывоз в <ADDRE...
3263556    ATP - 102\n\nАлюминиевый paдиaтop двигaтеля ув...
Name: description, Length: 201538, dtype: object

In [286]:
# from transformers import BertTokenizer, BertModel
# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')

In [287]:
# tokenizer('Sakura SASP377XE Кapтpидж cовмеcтимый Сaкуpa SP-377XE чеpный', return_length=True)['length']

In [ ]:
def color_eng(text):
    out = []
    for c in text:
        if c.lower() in 'qwertyuiopasdfghjklzxcvbnm':
            out.append('\x1b[0;30;41m' + c + '\x1b[0m')
        else:
            out.append(c)
    print(''.join(out))
    
color_eng(df_desc_eng.iloc[0])

iPhone 11 64gb

Емкоcть АКБ 87%

Фейc paботaет 
Полноcтью cоответcтвует фотогpaфиям.

🔥 Рaccpочкa - кpедит, от 18 

👉без пеpвонaчaльного взноca 

👉без официaльной paботы.

🍎 СКУПКА РЕМОНТ ОБМЕН

🔥Отпpaвляем по вcей Роccии, доcтaвкa aвито или любым удобным cпоcобом.

👍Имеетcя оптовaя пpодaжa, 

🥳В нaличие кучa aкcеccуapов: пеpеходник / шнуp / блок / зapядкa / aукc / aux / флешкa / и т.д.


In [ ]:
MAPPING_EN_RU = {
    'e': 'е',
    'y': 'у',
    'o': 'о',
    'p': 'р',
    'a': 'а',
    'k': 'к',
    'x': 'х',
    'c': 'с',
    'E': 'Е',
    'T': 'Т',
    'O': 'О',
    'P': 'Р',
    'A': 'А',
    'H': 'Н',
    'K': 'К',
    'X': 'Х',
    'C': 'С',
    'B': 'В',
    'M': 'М',
}
MAPPING_RU_EN = {v: k for k, v in MAPPING_EN_RU.items()}

import kenlm

model = kenlm.Model('../../kenlm/char_5gram_ru_en.bin')

import re

has_eng = re.compile('[a-zA-Z]')
r1 = re.compile(r'\b[a-zA-Zа-яА-ЯёЁ]+\b')
r2 = re.compile(r'[a-zA-Z]+')
r3 = re.compile(r'[а-яА-ЯёЁ]+')

def fix_text_shit(
    text, 
    lookaround=2, 
    # pad_thr=4, 
    skip_less_than=3,
):
    if not has_eng.match(text):
        return text
    ret_text = [i for i in text]
    for match in r1.finditer(text):
        # print(match, text[match.span()[1]:match.span()[0]])
        # print(re.fullmatch('[а-яА-ЯёЁ]+', text[match.span()[1]:match.span()[0]]))
        if match.span()[1] - match.span()[0] < skip_less_than:
            continue # pass if word too small
        if r2.fullmatch(text[match.span()[0]:match.span()[1]]):
            continue # pass if only eng
        if r3.fullmatch(text[match.span()[0]:match.span()[1]]):
            continue # pass if only ru
        slice_ = slice(match.span()[0] - lookaround, match.span()[1] + lookaround)
        # slice_ = slice(match.span()[0] - lookaround, match.span()[1] + lookaround) if match.span()[1] - match.span()[0] > pad_thr else slice(match.span()[0] - lookaround - 1, match.span()[1] + lookaround + 1)
        splitted = [i for i in text[slice_].lower()]
        splitted_ru = ''.join([MAPPING_EN_RU.get(i, i) for i in splitted])
        splitted_en = ''.join([MAPPING_RU_EN.get(i, i) for i in splitted])
        ppl_ru = model.perplexity(tok(splitted_ru))
        ppl_en = model.perplexity(tok(splitted_en))
        # ppl_ru = ppl(splitted_ru)
        # ppl_en = ppl(splitted_en)
        # print(splitted_ru, 'vs', splitted_en)
        if ppl_ru < ppl_en:
            # print('-- conflict resolved towards ru', match, ppl_ru, ppl_en)
            ret_text[match.span()[0]:match.span()[1]] = [MAPPING_EN_RU.get(i, i) for i in text[match.span()[0]:match.span()[1]]]
        elif ppl_ru > ppl_en:
            # print('-- conflict resolved towards en', match, ppl_ru, ppl_en)
            ret_text[match.span()[0]:match.span()[1]] = [MAPPING_RU_EN.get(i, i) for i in text[match.span()[0]:match.span()[1]]]
        else:
            # print('-- leave as is', match, ppl_ru, ppl_en)
            continue # leave as is
        
    res = ''.join(ret_text)
    res = res.replace('нa', 'на').replace(' c ', ' с ').replace(
        ' cо ', ' со ').replace(' сo ', ' со ').replace(
        ' co ', ' со ').replace('вo', 'во').replace(
        ' кo ', ' ко ').replace(' o ', ' о ').replace(
        'oб', 'об').replace('oт', 'от').replace(
        'зa', 'за').replace('пo', 'по').replace(
        'дo', 'до').replace(' y ', ' у ').replace(
        'cм', 'см').replace('гp', 'гр').replace(
        'пpo', 'про').replace('дa', 'да').replace(
        'нe', 'не').replace('тo', 'то').replace(
        'жe', 'же').replace('pyб', 'руб').replace(
        'eд', 'ед').replace('oна', 'она').replace(
        'онa', 'она').replace('oнa', 'она').replace(
        'oн', 'он').replace('eго', 'его').replace(
        'егo', 'его').replace('eгo', 'его').replace(
        'ниx', 'них').replace('иx', 'их').replace(
        'вcе', 'все').replace('всe', 'все').replace('вce', 'все')
        
    return res

In [298]:
test_text = df_desc_eng.iloc[4]
color_eng(test_text)

Sakura SASP377XE Кapтpидж cовмеcтимый Сaкуpa SP-377XE чеpный Black 6.4K для Ricoh Aficio SP-377DNwX SP-377, SP-377SFNwX

Сaмовывоз в <ADDRESS-#0>, доcтaвкa по <ADDRESS-#1> и <ADDRESS-#2>.

Пpедоплaтa 100%. Вapиaнты оплaты: кapтой, пеpеводом, QR кодом

Ценa c НДС. Полный комплект документов.

Нaличие нa 15-02-2024 08:34 - 100 шт. отгpузкa чеpез 2 дня;

Хapaктеpиcтики:
			
— IL: 1370574

			— Тип: Кapтpидж
— Тип чеpнил: Laser
— Сеpия: SP 377
— Номеp: SP-377XE
— Цвет: чеpный
— Совмеcтимоcть: Ricoh Aficio SP-377DNwX SP-377, SP-377SFNwX
— Реcуpc: 6400 cтp


In [299]:
color_eng(fix_text_shit(test_text, lookaround=2, skip_less_than=3))

Sakura SASP377XE Картридж совместимый Сакура SP-377XE черный Black 6.4K для Ricoh Aficio SP-377DNwX SP-377, SP-377SFNwX

Самовывоз в <ADDRESS-#0>, доставка по <ADDRESS-#1> и <ADDRESS-#2>.

Предоплата 100%. Варианты оплаты: картой, переводом, QR кодом

Цена с НДС. Полный комплект документов.

Наличие на 15-02-2024 08:34 - 100 шт. отгрузка через 2 дня;

Характеристики:
			
— IL: 1370574

			— Тип: Картридж
— Тип чернил: Laser
— Серия: SP 377
— Номер: SP-377XE
— Цвет: черный
— Совместимость: Ricoh Aficio SP-377DNwX SP-377, SP-377SFNwX
— Ресурс: 6400 cтp


In [300]:
%timeit fix_text_shit(test_text)

401 µs ± 3.96 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [304]:
sum(df_desc_eng_fixed != df['description'])

197524

In [305]:
sum(df_name_eng_fixed != df['name'])

131085

In [301]:
from tqdm import tqdm
tqdm.pandas()

In [ ]:
# def fix(x): 
#     return fix_text_shit(x) if has_eng.match(x) else x

df_desc_eng_fixed = df['description'].progress_apply(fix_text_shit)
df_name_eng_fixed = df['name'].progress_apply(fix_text_shit)

100%|████████████████████████████████████████████████████████████████████| 3263574/3263574 [00:11<00:00, 285598.07it/s]


In [279]:
df['description_fixed'] = df_desc_eng_fixed
df['name_fixed'] = df_name_eng_fixed

In [281]:
df = df[['name_fixed', 'description_fixed']]
df

,name_fixed,description_fixed
0,Детcкие cкaзки,Пpодaм по 50 p шт
1,Тpaнcфоpмaтоp тcл 100/10/6 квa б/у,Сухие тpaнcфоpмaтоpы 100-10 типa ТСЛ облaдaют ...
2,Ковaные диcки Rush R21 Ultimate Series R0990,В пpодaже комплект ковaных оpигинaльных диcков...
3,Рaзветвитель пpикуpивaтеля,"Рaзветвитель пpикуpивaтеля в нaличии 3 штуки,..."
4,Фapa пеpедняя LF7003 ASP,"Бpенд: ASP, apтикул: LF7003, Фapa пеpедняя. \n..."
...,...,...
3263569,Дивaн лондон,- НЕ БЕРЕМ ПРЕДОПЛАТУ - ОПЛАТА ПО ФАКТУ\n\n- Д...
3263570,Блузкa женcкaя 44 46,"Кpacивый блузон доpогой мapки, выглядит потpяc..."
3263571,Сумкa нoвaя,"Нoвaя женcкaя cумкa c pучкaми, еcть pемешoк!!!"
3263572,Темapи. Ручнaя вышивкa. Тaлиcмaн нa удaчу,Темapи- дpевнее япoнcкoе иcкуccтвo. Ручнaя выш...


In [282]:
df.to_parquet('names_descs_avito_fixed.parquet')